In [17]:
import math

# 1. ==================================
# Capacidades fixas pra ilustrar paralelismo
sectors = ["A", "B", "C", "D"]
capacity = {"A": 2, "B": 2, "C": 2, "D": 2}

# 2. ==================================
# Tarefas com blocos de 16h em A/B, 8h em C, 8h em D
tasks = [
    {"Tarefa": "T1", "A": 160, "B": 160, "C": 80, "D": 10},
    {"Tarefa": "T2", "A": 16, "B": 160, "C": 10, "D": 80},
    {"Tarefa": "T3", "A": 10,  "B": 160, "C": 160, "D": 80},
    {"Tarefa": "T4", "A": 80,  "B":  80, "C": 160, "D": 160},
]

# Dependências:
# - C só depois de A e B
# - D só depois de C
dependencies = {
    "C": ["A", "B"],
    "D": ["C"]
}

# 3. SIMULAÇÃO OTIMIZADA (com recursos e dependências)
for t in tasks:
    t["restante"] = {s: t[s] for s in sectors}
    t["opt_finish_by_sector"] = {}
    t["opt_finish"] = None

day = 0
while True:
    if all(t["restante"][s] == 0 for t in tasks for s in sectors):
        break
    day += 1
    used = {s: 0 for s in sectors}

    for s in sectors:
        # tarefas que ainda precisam deste setor e já cumpriram pré-requisitos
        ready = [
            t for t in tasks
            if t["restante"][s] > 0
               and all(t["restante"].get(dep, 0) == 0 for dep in dependencies.get(s, []))
        ]
        # dá prioridade às que têm menos hora neste setor
        ready.sort(key=lambda t: t["restante"][s])

        for t in ready:
            if used[s] >= capacity[s]:
                break
            work = min(8, t["restante"][s])
            t["restante"][s] -= work
            used[s] += 1
            if t["restante"][s] == 0:
                t["opt_finish_by_sector"][s] = day
                if all(t["restante"][x] == 0 for x in sectors):
                    t["opt_finish"] = day

# 4. BASELINE SEQUENCIAL (somando horas em cada setor, sem paralelismo nem dependências entre setores)
for t in tasks:
    total_hours = sum(t[s] for s in sectors)
    t["seq_finish"] = math.ceil(total_hours / 8)

# 5. IMPRESSÃO FINAL
print("Comparação por tarefa:")
for t in tasks:
    print(f"  Tarefa {t['Tarefa']}:")
    print(f"    otimizado por setor {t['opt_finish_by_sector']}, término final dia {t['opt_finish']}")
    print(f"    sequencial (sumário) = dia {t['seq_finish']}")


Comparação por tarefa:
  Tarefa T1:
    otimizado por setor {'B': 20, 'A': 22, 'C': 31, 'D': 32}, término final dia 32
    sequencial (sumário) = dia 52
  Tarefa T2:
    otimizado por setor {'A': 2, 'B': 30, 'C': 31, 'D': 40}, término final dia 40
    sequencial (sumário) = dia 34
  Tarefa T3:
    otimizado por setor {'A': 2, 'B': 40, 'C': 59, 'D': 68}, término final dia 68
    sequencial (sumário) = dia 52
  Tarefa T4:
    otimizado por setor {'B': 10, 'A': 12, 'C': 33, 'D': 52}, término final dia 52
    sequencial (sumário) = dia 60


In [16]:
import math
import pandas as pd

# --- 1. capacidade por setor (leia do seu CSV normalmente) ---
employees_df = pd.read_csv('funcionarios.csv')
sectors = ["A", "B", "C", "D"]
capacity = {sec: employees_df['Setores'].str.contains(sec).sum() for sec in sectors}

# 2. ==================================
# Tarefas com blocos de 16h em A/B, 8h em C, 8h em D
tasks = [
    {"Tarefa": "T1", "A": 160, "B": 160, "C": 80, "D": 10},
    {"Tarefa": "T2", "A": 16, "B": 160, "C": 10, "D": 80},
    {"Tarefa": "T3", "A": 10,  "B": 160, "C": 160, "D": 80},
    {"Tarefa": "T4", "A": 80,  "B":  80, "C": 160, "D": 160},
]

# Dependências:
# - C só depois de A e B
# - D só depois de C
dependencies = {
    "C": ["A", "B"],
    "D": ["C"]
}

# --- 3. simulação OTIMIZADA (com recursos e dependências) ---
for t in tasks:
    t["restante"] = {sec: t[sec] for sec in sectors}
    t["setor_conclusao_opt"] = {}     # vai guardar o dia em que cada setor termina
    t["conclusao_opt"] = None

dia = 0
while True:
    if all(t["restante"][sec] == 0 for t in tasks for sec in sectors):
        break
    dia += 1
    usados = {sec: 0 for sec in sectors}

    for sec in sectors:
        elegiveis = [
            t for t in tasks
            if t["restante"][sec] > 0
               and all(t["restante"].get(dep, 0) == 0 for dep in dependencies.get(sec, []))
        ]
        elegiveis.sort(key=lambda t: t["restante"][sec])

        for t in elegiveis:
            if usados[sec] >= capacity[sec]:
                break
            trab = min(8, t["restante"][sec])
            t["restante"][sec] -= trab
            usados[sec] += 1

            if t["restante"][sec] == 0:
                t["setor_conclusao_opt"][sec] = dia
                if all(t["restante"][s] == 0 for s in sectors):
                    t["conclusao_opt"] = dia

# --- 4. cálculo SEQUENCIAL (caminho crítico interno, sem limitação de recursos) ---
def compute_sequential(t):
    memo = {}
    def finish(sec):
        if t[sec] == 0:
            return 0.0
        if sec in memo:
            return memo[sec]
        prev = 0.0
        for p in dependencies.get(sec, []):
            prev = max(prev, finish(p))
        dur = t[sec] / 8.0
        memo[sec] = prev + dur
        return memo[sec]

    setor_finish = {}
    for sec in sectors:
        if t[sec] > 0:
            setor_finish[sec] = math.ceil(finish(sec))
    final = max(setor_finish.values(), default=0)
    return setor_finish, final

for t in tasks:
    seq_setor, seq_final = compute_sequential(t)
    t["setor_conclusao_seq"] = seq_setor
    t["conclusao_seq"] = seq_final

# --- 5. imprimir RESULTADOS ---
print("Comparação por tarefa:")
for t in tasks:
    print(f"  Tarefa {t['Tarefa']}:")
    print(f"    otimizado por setor {t['setor_conclusao_opt']}, término final dia {t['conclusao_opt']}")
    print(f"    sequencial por setor {t['setor_conclusao_seq']}, término final dia {t['conclusao_seq']}")


Comparação por tarefa:
  Tarefa T1:
    otimizado por setor {'A': 20, 'B': 20, 'C': 29, 'D': 30}, término final dia 30
    sequencial por setor {'A': 20, 'B': 20, 'C': 30, 'D': 32}, término final dia 32
  Tarefa T2:
    otimizado por setor {'A': 2, 'B': 20, 'C': 21, 'D': 30}, término final dia 30
    sequencial por setor {'A': 2, 'B': 20, 'C': 22, 'D': 32}, término final dia 32
  Tarefa T3:
    otimizado por setor {'A': 2, 'B': 20, 'C': 39, 'D': 48}, término final dia 48
    sequencial por setor {'A': 2, 'B': 20, 'C': 40, 'D': 50}, término final dia 50
  Tarefa T4:
    otimizado por setor {'A': 10, 'B': 10, 'C': 29, 'D': 48}, término final dia 48
    sequencial por setor {'A': 10, 'B': 10, 'C': 30, 'D': 50}, término final dia 50


In [19]:
import math
import pandas as pd

# --- 1. Capacidade por setor (leia do CSV) ---
employees_df = pd.read_csv('funcionarios.csv')
sectors = ["A", "B", "C", "D"]
capacity = {sec: employees_df['Setores'].str.contains(sec).sum() for sec in sectors}

# --- 2. Definição das tarefas e dependências ---
tasks = [
    {"Tarefa": "T1", "A": 160, "B": 160, "C": 80, "D": 10},
    {"Tarefa": "T2", "A": 16, "B": 160, "C": 10, "D": 80},
    {"Tarefa": "T3", "A": 10, "B": 160, "C": 160, "D": 80},
    {"Tarefa": "T4", "A": 80, "B": 80, "C": 160, "D": 160},
]

dependencies = {
    "C": ["A", "B"],
    "D": ["C"]
}

# --- 3. Simulação OTIMIZADA (com recursos e dependências) ---
for t in tasks:
    t["restante"] = {sec: t[sec] for sec in sectors}
    t["setor_conclusao_opt"] = {}
    t["conclusao_opt"] = None

dia = 0
tarefas_concluidas = [False] * len(tasks)  # Rastrear conclusão por tarefa

while not all(tarefas_concluidas):
    dia += 1
    usados = {sec: 0 for sec in sectors}

    for sec in sectors:
        # Lista de tarefas elegíveis para o setor
        elegiveis = [
            t for idx, t in enumerate(tasks)
            if not tarefas_concluidas[idx] and
               t["restante"][sec] > 0 and
               all(t["restante"].get(dep, 0) == 0 for dep in dependencies.get(sec, []))
        ]
        elegiveis.sort(key=lambda t: t["restante"][sec])

        for t in elegiveis:
            if usados[sec] >= capacity[sec]:
                break

            # Trabalhar no setor (máx 8h/dia por funcionário)
            trab = min(8, t["restante"][sec])
            t["restante"][sec] -= trab
            usados[sec] += 1

            # Registrar conclusão do setor se necessário
            if t["restante"][sec] == 0:
                t["setor_conclusao_opt"][sec] = dia

    # Verificar se a tarefa foi concluída (todos os setores)
    for idx, t in enumerate(tasks):
        if not tarefas_concluidas[idx] and all(rest == 0 for rest in t["restante"].values()):
            t["conclusao_opt"] = dia
            tarefas_concluidas[idx] = True

# --- 4. Cálculo SEQUENCIAL (caminho crítico, sem recursos) ---
def compute_sequential(t):
    memo = {}
    
    def finish(sec):
        if sec in memo:
            return memo[sec]
        
        prev = 0.0
        for dep in dependencies.get(sec, []):
            prev = max(prev, finish(dep))
        
        # Calcular duração (ignorar setor com 0 horas)
        dur = math.ceil(t[sec] / 8.0) if t[sec] > 0 else 0.0
        memo[sec] = prev + dur
        return memo[sec]
    
    setor_finish = {}
    for sec in sectors:
        if t[sec] > 0:
            setor_finish[sec] = math.ceil(finish(sec))
    
    final = max(setor_finish.values()) if setor_finish else 0
    return setor_finish, final

for t in tasks:
    seq_setor, seq_final = compute_sequential(t)
    t["setor_conclusao_seq"] = seq_setor
    t["conclusao_seq"] = seq_final

# --- 5. Imprimir RESULTADOS ---
print("Comparação por tarefa:")
for t in tasks:
    print(f"  Tarefa {t['Tarefa']}:")
    print(f"    Otimizado: Setores {t['setor_conclusao_opt']}, Término dia {t['conclusao_opt']}")
    print(f"    Sequencial: Setores {t['setor_conclusao_seq']}, Término dia {t['conclusao_seq']}")

Comparação por tarefa:
  Tarefa T1:
    Otimizado: Setores {'A': 20, 'B': 20, 'C': 29, 'D': 30}, Término dia 30
    Sequencial: Setores {'A': 20, 'B': 20, 'C': 30, 'D': 32}, Término dia 32
  Tarefa T2:
    Otimizado: Setores {'A': 2, 'B': 20, 'C': 21, 'D': 30}, Término dia 30
    Sequencial: Setores {'A': 2, 'B': 20, 'C': 22, 'D': 32}, Término dia 32
  Tarefa T3:
    Otimizado: Setores {'A': 2, 'B': 20, 'C': 39, 'D': 48}, Término dia 48
    Sequencial: Setores {'A': 2, 'B': 20, 'C': 40, 'D': 50}, Término dia 50
  Tarefa T4:
    Otimizado: Setores {'A': 10, 'B': 10, 'C': 29, 'D': 48}, Término dia 48
    Sequencial: Setores {'A': 10, 'B': 10, 'C': 30, 'D': 50}, Término dia 50
